In [1]:
from IPython.display import Image, display
from glob import glob
from tqdm import tqdm
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import inception_v3

In [2]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=True)
model._layers[0].batch_input_shape = (None, None, None, 3)
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [3]:
def preprocess_image(img):
    """ Preprocess image for inception_v3 """
    
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = inception_v3.preprocess_input(img)
    
    return img

def deprocess_image(x):
    """ Deprocess image from inception_v3 tensor """
    
    x = x.reshape((x.shape[1], x.shape[2], 3))
    # Undo inception_v3 preprocess
    x /= 2.
    x += 0.5
    x *= 255

    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

In [4]:
def compute_loss(input_image, feature_extractor, layer_settings):
    """
    Computes loss based on a feature extractor and the corresponding layers weights.
    
    Parameters:
        input_image (np.ndarray): numpy array of an image.
        feature_extractor (tf.Model): feature extractor with outputs corresponding to the layers.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        
    Returns:
        loss (float): loss of the image w.r.t. the layers specified in layer_settings.
    """
    
    # Computes features from image
    features = feature_extractor(input_image)
    
    loss = tf.zeros(shape=())
    for key in features.keys():
        # Coefficient of the layer
        coeff = layer_settings[key]
        # Activation of the layer
        activation = features[key]
        # Adds to loss (avoid artifacts by removing borders)
        scaling = tf.reduce_prod(tf.cast(tf.shape(activation), 'float32'))
        #loss += coeff * tf.reduce_sum(tf.square(activation[:,2:-2,2:-2,:])) / scaling
        loss += coeff * tf.reduce_sum(tf.square(activation)) / scaling
    
    return loss

@tf.function
def gradient_ascent_step(img, feature_extractor, layer_settings, learning_rate):
    """
    Performs a gradient ascent step on an image.
    
    Parameters:
        img (np.ndarray): numpy array of an image.
        feature_extractor (tf.Model): feature extractor with outputs corresponding to the layers.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        learning_rate (float): learning rate for the gradient ascent step.
        
    Returns:
        loss (float32): loss of the image w.r.t. the layers specified in layer_settings.
        img (np.ndarray): numpy array of the modified image
    """
    
    # Computes loss with GradientTape
    with tf.GradientTape() as tape:
        tape.watch(img)
        loss = compute_loss(img, feature_extractor, layer_settings)
    # Computes gradients and normalize
    grads = tape.gradient(loss, img)
    grads /= tf.maximum(tf.reduce_mean(tf.abs(grads)), 1e-6)
    # Gradient ascent step
    img += learning_rate * grads
    
    return loss, img

def gradient_ascent_loop(img, feature_extractor, layer_settings, iterations, learning_rate, octave, shape, destination_filepath=None, max_loss=None):
    """
    Performs the gradient ascent loop on an image.
    
    Parameters:
        img (np.ndarray): numpy array of an image.
        feature_extractor (tf.Model): feature extractor with outputs corresponding to the layers.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        iterations (int): number of iterations for the loop.
        learning_rate (float): learning rate for the gradient ascent steps.
        max_loss (float): maximum loss before interruption (default=None).
        
    Returns:
        img (np.ndarray): numpy array of the modified image
    """
    
    # gradient ascent loop
    losses = []
    t = tqdm(range(iterations))
    for i in t:
        loss, img = gradient_ascent_step(img, feature_extractor, layer_settings, learning_rate)
        losses.append(loss)
        
        description = ""
        if destination_filepath:
            description += destination_filepath + " - "
        description += "Octave: {0:d}, Shape: {1:s}, Loss: [{2:.2f}, {3:.2f}]".format(octave, str(shape), np.min(losses), np.max(losses))
        t.set_description(description)
        t.refresh()
        if max_loss is not None and loss > max_loss:
            break
        #print("Loss at step %d: %.2f" % (i, loss))
    #print("Min loss: %.2f - Max loss: %.2f" % (np.min(losses), np.max(losses)))
    return img

In [5]:
def dreamify(source_filepath,
             destination_filepath,
             model,
             layer_settings,
             learning_rate = 0.01,
             num_octave = 3,
             octave_scale = 1.5,
             iterations = 20,
             max_loss = None):
    """
    Dreamifies an image.
    Returns nothing, the image is automatically saved at the requested destination.
    
    Parameters:
        image_path (str): path to the image folder (must end with "/").
        source_filename (str): name of the original file (with file format).
        destination_filename (str): name of the destination file (without file format).
        model (tf.Model): model to be used.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        learning_rate (float): learning rate for the gradient ascent steps (deault=0.01).
        num_octave (int): number of subsampling octaves (default=3).
        octave_scale (float): scale of each subsampling (deault=1.5).
        iterations (int): number of iterations for the gradient ascent loop at each scale (deafault=20).
        max_loss (float): maximum loss before interruption on a loop (default=15.0).
    """
    
    # Dict of output layers
    outputs_dict = dict([(layer.name, layer.output)
                         for layer in [model.get_layer(key)
                                       for key in layer_settings.keys()]])
    # Feature extractor from model input layer and dict of output layers
    feature_extractor = keras.Model(inputs = model.inputs,
                                    outputs = outputs_dict)
    
    # Loads image, preprocesses it and gets its shape
    original_img = keras.preprocessing.image.load_img(source_filepath)
    original_img = preprocess_image(original_img)
    original_shape = original_img.shape[1:3]
    
    # Creates the list of successive shapes to use
    successive_shapes = [original_shape]
    for i in range(1, num_octave):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)
    successive_shapes = successive_shapes[::-1] # Invert
    
    # Image at minimum shape
    shrunk_original_img = tf.image.resize(original_img, successive_shapes[0])
    # Image to be modified
    img = tf.identity(original_img)
    
    for i, shape in enumerate(successive_shapes):
        #print("Octave %d with shape %s" % (i, shape))
        # Resizes at current shape
        img = tf.image.resize(img, shape)
        # Performs the gradient ascent loop on the current shape
        img = gradient_ascent_loop(img, 
                                   feature_extractor, 
                                   layer_settings,
                                   iterations=iterations,
                                   learning_rate=learning_rate,
                                   octave=i,
                                   shape=shape,
                                   destination_filepath=destination_filepath,
                                   max_loss=max_loss)
        # Restores lost details
        upscaled_shrunk_original_img = tf.image.resize(shrunk_original_img, shape)
        same_size_original = tf.image.resize(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail
        # Resizes the minimum shape to the current shape for next loop
        shrunk_original_img = tf.image.resize(original_img, shape)
        
    # Stores the resulting image
    keras.preprocessing.image.save_img(destination_filepath + '.png', deprocess_image(img.numpy()))

In [15]:
def single_layer_dreamify(source_filepath, layer_index, coeff, max_loss=None, display_image=False):
    """ 
    Dreamifies an image using a single layer.
    Returns nothing, the image is automatically saved at the requested destination.
    
    Parameters:
        source_filepath (str) = filepath of the source file.
        layer_index (int) = index of the layer to use.
        coeff (float) = coefficient of the layer in the layer_settings dict.
        max_loss (float) = maximum loss before breaking loop (default=None).
        display_image (bool) = display the result (default=False).
    """
    
    layer_settings = {model.layers[layer_index].name : coeff}
    
    destination_filepath = source_filepath[:-4] + "->" + str(layer_index) + "=" + str(np.around(coeff, decimals=2))
    
    dreamify(source_filepath, destination_filepath, model, layer_settings, max_loss=max_loss)
    
    if display_image:
        display(Image(destination_filepath + '.png'))
        
    return destination_filepath + '.png'

In [16]:
def generate_dreams(file_path, depth, n_images, seed=0, max_coeff=10, max_loss=20):
    
    np.random.seed(seed)
    
    for n in range(n_images):
        
        image_path = file_path
        
        for d in range(depth):

            layer = np.random.randint(0, len(model.layers))
            coeff = np.random.uniform(high=max_coeff)
            
            image_path = single_layer_dreamify(image_path, layer, coeff, max_loss)

In [17]:
path = "Images/Personal-Propic/o.jpg"
depth = 3
n_images = 10

generate_dreams(file_path = path, depth = depth, n_images = n_images)

Images/Personal-Propic/o->172=5.93 - Octave: 0, Shape: (382, 382), Loss: [4.42, 21.35]:  40%|████      | 8/20 [00:05<00:08,  1.38it/s]
Images/Personal-Propic/o->172=5.93 - Octave: 1, Shape: (573, 573), Loss: [5.18, 20.37]:  25%|██▌       | 5/20 [00:08<00:24,  1.66s/it]
Images/Personal-Propic/o->172=5.93 - Octave: 2, Shape: (860, 860), Loss: [4.48, 20.41]:  25%|██▌       | 5/20 [00:18<00:54,  3.62s/it]
Images/Personal-Propic/o->172=5.93->192=6.03 - Octave: 0, Shape: (382, 382), Loss: [1.82, 20.52]:  70%|███████   | 14/20 [00:10<00:04,  1.39it/s]
Images/Personal-Propic/o->172=5.93->192=6.03 - Octave: 1, Shape: (573, 573), Loss: [1.77, 21.11]:  65%|██████▌   | 13/20 [00:19<00:10,  1.52s/it]
Images/Personal-Propic/o->172=5.93->192=6.03 - Octave: 2, Shape: (860, 860), Loss: [1.48, 21.51]:  70%|███████   | 14/20 [00:47<00:20,  3.39s/it]
Images/Personal-Propic/o->172=5.93->192=6.03->195=8.47 - Octave: 0, Shape: (382, 382), Loss: [4.41, 20.18]:  20%|██        | 4/20 [00:04<00:18,  1.16s/it]
Im

In [7]:
path = "Images/Personal-Propic/"
iterations = 3
n_layers = 4
max_coeff = 10
max_loss = 20
seed = 0

np.random.seed(seed)

for i in range(iterations):
    
    paths = [p for p in (glob(path + "*.png") + glob(path + "*.jpg"))]
    
    for source_path in paths:
        
        layers = np.random.randint(0, len(model.layers), n_layers)

        for l in layers:
            
            coeff = np.random.uniform(high=max_coeff)
            single_layer_dreamify(source_path, l, coeff, max_loss)

Images/Personal-Propic/o->172=6.03 - Octave: 0, Shape: (382, 382), Loss: [4.49, 20.04]:  35%|███▌      | 7/20 [00:06<00:12,  1.02it/s]
Images/Personal-Propic/o->172=6.03 - Octave: 1, Shape: (573, 573), Loss: [5.18, 20.60]:  25%|██▌       | 5/20 [00:07<00:22,  1.52s/it]
Images/Personal-Propic/o->172=6.03 - Octave: 2, Shape: (860, 860), Loss: [4.65, 22.80]:  30%|███       | 6/20 [00:18<00:43,  3.11s/it]
Images/Personal-Propic/o->47=5.45 - Octave: 0, Shape: (382, 382), Loss: [9.63, 23.35]:  20%|██        | 4/20 [00:01<00:07,  2.13it/s]
Images/Personal-Propic/o->47=5.45 - Octave: 1, Shape: (573, 573), Loss: [11.35, 22.33]:  15%|█▌        | 3/20 [00:03<00:19,  1.12s/it]
Images/Personal-Propic/o->47=5.45 - Octave: 2, Shape: (860, 860), Loss: [12.06, 23.26]:  15%|█▌        | 3/20 [00:06<00:38,  2.27s/it]
Images/Personal-Propic/o->117=4.24 - Octave: 0, Shape: (382, 382), Loss: [1.53, 16.79]: 100%|██████████| 20/20 [00:09<00:00,  2.11it/s]
Images/Personal-Propic/o->117=4.24 - Octave: 1, Shape: 

KeyboardInterrupt: 